In [1]:
 %matplotlib inline
import pandas as pd 
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [2]:
def stringToInt(var):
    try:
        var = np.int16(var)
        return var
    except Exception:
        if var == 'R':
            return np.int16(300)# Setting a arbituary hight value for Forclosed home
        else:
            return np.int16(-1)

def stringToFloat(var):
    try:
        var = np.float32(var)
        return var
    except Exception:
        return np.float32(0)

def fillFlag(var):
    try:
        var=np.char(var)
        return var
    except Exception:
        return np.char('F')


### Data file containing loan origination information for Year 2010

In [3]:
fields_Origin = ['CREDIT SCORE','FIRST TIME HOMEBUYER FLAG','MORTGAGE INSURANCE PERCENTAGE','CLTV','DTI Ratio',
          'ORIGINAL UPB','ORIGINAL LTV','ORIGINAL INTEREST RATE' ,'LOAN SEQUENCE NUMBER']    #9
fData=pd.read_csv('data/historical_data1_Q12010/historical_data1_Q12010.txt','|',index_col=None, encoding='utf-8',low_memory=False
                 ,usecols=fields_Origin,converters={'CREDIT SCORE':stringToInt,
                                                     'DTI Ratio':stringToFloat})
len(fData)

261882

### Data file containing loan monthly data for Year 2010. 

In [4]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y%m')
fields_Month=['LOAN SEQUENCE NUMBER','MONTHLY REPORTING PERIOD','CURRENT ACTUAL UPB','CURRENT LOAN DELINQUENCY STATUS',
              'REMAINING MONTHS TO LEGAL MATURITY','REPURCHASE FLAG','MODIFICATION FLAG',
              'ZERO BALANCE CODE','ZERO BALANCE EFFECTIVE DATE','CURRENT INTEREST RATE','CURRENT DEFERRED UPB']
reader=pd.read_csv('data/historical_data1_Q12010/historical_data1_time_Q12010.txt','|',
                   index_col=None, parse_dates=['MONTHLY REPORTING PERIOD'],
                   date_parser=dateparse,encoding='utf-8',low_memory=False,chunksize=10000
                  ,usecols=fields_Month,converters={'CURRENT ACTUAL UPB':stringToFloat,
                                                    'CURRENT LOAN DELINQUENCY STATUS':stringToInt})


#Get dataframe from chunks
df = pd.concat(chunk for chunk in reader)

#Fill missing data
df['REPURCHASE FLAG'].fillna('F', inplace=True)
df['MODIFICATION FLAG'].fillna('N', inplace=True)
df['ZERO BALANCE CODE'].fillna('00', inplace=True)
df['ZERO BALANCE EFFECTIVE DATE'].fillna('000000', inplace=True)

df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 11511298 entries, 0 to 1297
Data columns (total 11 columns):
LOAN SEQUENCE NUMBER                  object
MONTHLY REPORTING PERIOD              datetime64[ns]
CURRENT ACTUAL UPB                    float64
CURRENT LOAN DELINQUENCY STATUS       int64
REMAINING MONTHS TO LEGAL MATURITY    int64
REPURCHASE FLAG                       object
MODIFICATION FLAG                     object
ZERO BALANCE CODE                     object
ZERO BALANCE EFFECTIVE DATE           object
CURRENT INTEREST RATE                 float64
CURRENT DEFERRED UPB                  float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 1.0+ GB


In [5]:
df.head()

,LOAN SEQUENCE NUMBER,MONTHLY REPORTING PERIOD,CURRENT ACTUAL UPB,CURRENT LOAN DELINQUENCY STATUS,REMAINING MONTHS TO LEGAL MATURITY,REPURCHASE FLAG,MODIFICATION FLAG,ZERO BALANCE CODE,ZERO BALANCE EFFECTIVE DATE,CURRENT INTEREST RATE,CURRENT DEFERRED UPB
0,F110Q1000001,2010-02-01,0.0,0,360,F,N,00,000000,4.875,0.0
1,F110Q1000001,2010-03-01,0.0,0,359,F,N,00,000000,4.875,0.0
2,F110Q1000001,2010-04-01,0.0,0,358,F,N,00,000000,4.875,0.0
3,F110Q1000001,2010-05-01,0.0,0,357,F,N,00,000000,4.875,0.0
4,F110Q1000001,2010-06-01,0.0,0,356,F,N,00,000000,4.875,0.0


In [6]:
df2=df.sort_values(by='CURRENT LOAN DELINQUENCY STATUS',ascending=False)
print (df2['MONTHLY REPORTING PERIOD'].max())
print (df2['MONTHLY REPORTING PERIOD'].min())

2015-12-01 00:00:00
2010-02-01 00:00:00


In [7]:
#Calculate two new columns 
df['year']=df['MONTHLY REPORTING PERIOD'].dt.year
df['quarter']=df['MONTHLY REPORTING PERIOD'].dt.quarter
df.head()

,LOAN SEQUENCE NUMBER,MONTHLY REPORTING PERIOD,CURRENT ACTUAL UPB,CURRENT LOAN DELINQUENCY STATUS,REMAINING MONTHS TO LEGAL MATURITY,REPURCHASE FLAG,MODIFICATION FLAG,ZERO BALANCE CODE,ZERO BALANCE EFFECTIVE DATE,CURRENT INTEREST RATE,CURRENT DEFERRED UPB,year,quarter
0,F110Q1000001,2010-02-01,0.0,0,360,F,N,00,000000,4.875,0.0,2010,1
1,F110Q1000001,2010-03-01,0.0,0,359,F,N,00,000000,4.875,0.0,2010,1
2,F110Q1000001,2010-04-01,0.0,0,358,F,N,00,000000,4.875,0.0,2010,2
3,F110Q1000001,2010-05-01,0.0,0,357,F,N,00,000000,4.875,0.0,2010,2
4,F110Q1000001,2010-06-01,0.0,0,356,F,N,00,000000,4.875,0.0,2010,2


#### Removing Loan data with no credit score

In [8]:
print (len(fData))
fDataClean=fData[fData['CREDIT SCORE']>0]
print (len(fDataClean))

261882
261871


#### Removing invalid data (Missing CURRENT LOAN DELINQUENCY STATUS) and Forclosed data out.

In [9]:
print (len(df))
dfClean=df[df['CURRENT LOAN DELINQUENCY STATUS'] >-1]
print (len(dfClean))
dfClean=dfClean[dfClean['CURRENT LOAN DELINQUENCY STATUS'] <300]
print (len(dfClean))

11511298
11511297
11510224


### Merging  Loan Origination with Monthly datasets 

In [10]:

dfMonthly = pd.merge(dfClean,fDataClean,on='LOAN SEQUENCE NUMBER')
dfMonthly.head()
print (len(dfMonthly))

11509676


### Slicing and peeking into dataset

In [11]:
df1=dfMonthly[['LOAN SEQUENCE NUMBER','CREDIT SCORE','CURRENT LOAN DELINQUENCY STATUS']]
df1.head(20)

,LOAN SEQUENCE NUMBER,CREDIT SCORE,CURRENT LOAN DELINQUENCY STATUS
0,F110Q1000001,777,0
1,F110Q1000001,777,0
2,F110Q1000001,777,0
3,F110Q1000001,777,0
4,F110Q1000001,777,0
5,F110Q1000001,777,0
6,F110Q1000001,777,0
7,F110Q1000001,777,0
8,F110Q1000001,777,0
9,F110Q1000002,801,0


### Understanding key values

In [12]:
df2=df1.sort_values(by='CURRENT LOAN DELINQUENCY STATUS',ascending=False)
df2.head(20)

,LOAN SEQUENCE NUMBER,CREDIT SCORE,CURRENT LOAN DELINQUENCY STATUS
2488261,F110Q1055758,699,129
5588304,F110Q1128148,672,121
1369239,F110Q1030453,778,69
1369238,F110Q1030453,778,68
1369237,F110Q1030453,778,67
1369236,F110Q1030453,778,66
1369235,F110Q1030453,778,65
1369234,F110Q1030453,778,64
3623820,F110Q1081641,729,64
3531602,F110Q1079562,690,63


### Grouping data for each loan.  Total number of unique loans

In [13]:
df2=df1.groupby('LOAN SEQUENCE NUMBER').max()
df3=df2.reset_index()
df3.head()

len(df3)

261860

### Total of non performing loan

In [14]:
df4= df3[df3['CURRENT LOAN DELINQUENCY STATUS']>0]
len(df4)

17131

### Monthly GDP data from Labour Department

In [15]:
dateparse2 = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
fields_gdp=['date','change-current','change-chained']
gdpData= pd.read_csv('data/gdp-quarter.csv',',', parse_dates=['date'],date_parser=dateparse2,usecols=fields_gdp)
gdpData=gdpData[(gdpData.date > np.datetime64('2009-12-30')) & (gdpData.date < np.datetime64('2016-01-01'))]
gdpData['year']=gdpData['date'].dt.year
gdpData['quarter']=gdpData['date'].dt.quarter
gdpData

,date,change-current,change-chained,year,quarter
251,2010-01-01,3.0,1.6,2010,1
252,2010-04-01,5.8,3.9,2010,2
253,2010-07-01,4.7,2.8,2010,3
254,2010-10-01,4.9,2.8,2010,4
255,2011-01-01,0.3,-1.3,2011,1
256,2011-04-01,5.9,3.2,2011,2
257,2011-07-01,3.9,1.4,2011,3
258,2011-10-01,5.4,4.9,2011,4
259,2012-01-01,5.8,3.7,2012,1
260,2012-04-01,3.0,1.2,2012,2


### Merging the values with above dfMonthly dataset

In [16]:
dfWithG=pd.merge(dfMonthly,gdpData,on=['year','quarter'])


In [17]:
dfWithG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9417927 entries, 0 to 9417926
Data columns (total 24 columns):
LOAN SEQUENCE NUMBER                  object
MONTHLY REPORTING PERIOD              datetime64[ns]
CURRENT ACTUAL UPB                    float64
CURRENT LOAN DELINQUENCY STATUS       int64
REMAINING MONTHS TO LEGAL MATURITY    int64
REPURCHASE FLAG                       object
MODIFICATION FLAG                     object
ZERO BALANCE CODE                     object
ZERO BALANCE EFFECTIVE DATE           object
CURRENT INTEREST RATE                 float64
CURRENT DEFERRED UPB                  float64
year                                  int64
quarter                               int64
CREDIT SCORE                          int64
FIRST TIME HOMEBUYER FLAG             object
MORTGAGE INSURANCE PERCENTAGE         object
CLTV                                  float64
DTI Ratio                             float64
ORIGINAL UPB                          int64
ORIGINAL LTV             

### Getting the needed data from above merged dataset

In [18]:
dfWithG=dfWithG[['LOAN SEQUENCE NUMBER','MONTHLY REPORTING PERIOD','CURRENT ACTUAL UPB','CURRENT LOAN DELINQUENCY STATUS',
              'REMAINING MONTHS TO LEGAL MATURITY','REPURCHASE FLAG','MODIFICATION FLAG',
              'ZERO BALANCE CODE','CURRENT INTEREST RATE','CURRENT DEFERRED UPB','year','quarter',
              'CREDIT SCORE','FIRST TIME HOMEBUYER FLAG','MORTGAGE INSURANCE PERCENTAGE','CLTV','DTI Ratio',
              'change-current','change-chained']]

### Saving Data in pickle object onto the hard drive

In [19]:
dfWithG.to_pickle('data/dataframe.pkl')

### Testing the saved pickle object

In [20]:
dfWithG=pd.read_pickle('data/dataframe.pkl')
dfWithG.head()

,LOAN SEQUENCE NUMBER,MONTHLY REPORTING PERIOD,CURRENT ACTUAL UPB,CURRENT LOAN DELINQUENCY STATUS,REMAINING MONTHS TO LEGAL MATURITY,REPURCHASE FLAG,MODIFICATION FLAG,ZERO BALANCE CODE,CURRENT INTEREST RATE,CURRENT DEFERRED UPB,year,quarter,CREDIT SCORE,FIRST TIME HOMEBUYER FLAG,MORTGAGE INSURANCE PERCENTAGE,CLTV,DTI Ratio,change-current,change-chained
0,F110Q1000001,2010-02-01,0.0,0,360,F,N,00,4.875,0.0,2010,1,777,N,000,43.0,26.0,3.0,1.6
1,F110Q1000001,2010-03-01,0.0,0,359,F,N,00,4.875,0.0,2010,1,777,N,000,43.0,26.0,3.0,1.6
2,F110Q1000003,2010-02-01,0.0,0,360,F,N,00,4.875,0.0,2010,1,727,N,000,61.0,26.0,3.0,1.6
3,F110Q1000003,2010-03-01,0.0,0,359,F,N,00,4.875,0.0,2010,1,727,N,000,61.0,26.0,3.0,1.6
4,F110Q1000004,2010-03-01,0.0,0,360,F,N,00,4.875,0.0,2010,1,794,N,000,58.0,31.0,3.0,1.6
